In [1]:
import numpy as np
import pandas as pd
import os
import zipfile
import cv2
from matplotlib import pyplot as plt
import shutil
from tqdm import tqdm
import torch
import torchvision
import time
import copy
from torchvision import transforms, models

In [2]:
!ls -la

total 191592
drwx------+ 1 ruslan ruslan         0 Mar 14 15:42 .
drwx------+ 1 ruslan ruslan         0 Mar 10 18:07 ..
drwx------+ 1 ruslan ruslan         0 Mar 10 18:20 .ipynb_checkpoints
-rwx------+ 1 ruslan ruslan 196168345 Mar 14 15:42 baseline_in_pytorch.ipynb
drwx------+ 1 ruslan ruslan         0 Mar 13 22:59 plates
-rwx------+ 1 ruslan ruslan      9248 Mar 13 23:48 submission.csv
drwx------+ 1 ruslan ruslan         0 Mar 13 23:45 test
drwx------+ 1 ruslan ruslan         0 Mar 13 23:01 train
drwx------+ 1 ruslan ruslan         0 Mar 13 23:01 val


In [3]:
!python --version

Python 3.11.7


In [4]:
print(os.listdir("."))

['.ipynb_checkpoints', 'baseline_in_pytorch.ipynb', 'plates', 'submission.csv', 'test', 'train', 'val']


In [7]:
# with zipfile.ZipFile('D:/kpfu/magistracy/1_course_1_semester/fundamentals_of_technical_vision/курс по нейронкам степик/7.3/plates.zip', 'r') as zip_obj:
#    zip_obj.extractall(".")

# print('After zip extraction:')
# print(os.listdir("."))

In [3]:
data_root = "./plates"
print(os.listdir(data_root))

['test', 'train']


In [4]:
# for class_name in ['cleaned', 'dirty']:
#     source_dir = os.path.join(data_root, 'train', class_name)
#     os.remove(os.path.join(source_dir, '.DS_Store'))

In [5]:
train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

# for dir_name in [train_dir, val_dir]:
#     for class_name in class_names:
#         os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

# for class_name in class_names:
#     source_dir = os.path.join(data_root, 'train', class_name)
#     for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
#         if i % 6 != 0:
#             dest_dir = os.path.join(train_dir, class_name)
#         else:
#             dest_dir = os.path.join(val_dir, class_name)
#         shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [6]:
!ls train

cleaned
dirty


In [7]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

print(train_dataset)

batch_size = 8
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

Dataset ImageFolder
    Number of datapoints: 32
    Root location: train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [8]:
len(train_dataloader), len(train_dataset)
print(train_dataloader.dataset.samples)

[('train\\cleaned\\0001.jpg', 0), ('train\\cleaned\\0002.jpg', 0), ('train\\cleaned\\0003.jpg', 0), ('train\\cleaned\\0004.jpg', 0), ('train\\cleaned\\0005.jpg', 0), ('train\\cleaned\\0007.jpg', 0), ('train\\cleaned\\0008.jpg', 0), ('train\\cleaned\\0009.jpg', 0), ('train\\cleaned\\0010.jpg', 0), ('train\\cleaned\\0011.jpg', 0), ('train\\cleaned\\0013.jpg', 0), ('train\\cleaned\\0014.jpg', 0), ('train\\cleaned\\0015.jpg', 0), ('train\\cleaned\\0016.jpg', 0), ('train\\cleaned\\0017.jpg', 0), ('train\\cleaned\\0019.jpg', 0), ('train\\dirty\\0001.jpg', 1), ('train\\dirty\\0002.jpg', 1), ('train\\dirty\\0003.jpg', 1), ('train\\dirty\\0004.jpg', 1), ('train\\dirty\\0005.jpg', 1), ('train\\dirty\\0007.jpg', 1), ('train\\dirty\\0008.jpg', 1), ('train\\dirty\\0009.jpg', 1), ('train\\dirty\\0010.jpg', 1), ('train\\dirty\\0011.jpg', 1), ('train\\dirty\\0013.jpg', 1), ('train\\dirty\\0014.jpg', 1), ('train\\dirty\\0015.jpg', 1), ('train\\dirty\\0016.jpg', 1), ('train\\dirty\\0017.jpg', 1), ('trai

In [31]:
# X_batch, y_batch = next(iter(train_dataloader))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
# plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean);
# print(X_batch.shape)
# print(X_batch[0].shape)

In [32]:
def show_input(input_tensor, title='', num_batch=0, num_elem_batch=0):
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    # print(title)
    # print(image)
    plt.imshow(image.clip(0, 1))
    plt.title(str(num_batch) + "." + str(num_elem_batch) + ") " + title)
    plt.show()
    plt.pause(0.001)

# X_batch, y_batch = next(iter(train_dataloader))
# for x_item, y_item in zip(X_batch, y_batch):
#     show_input(x_item, title=class_names[y_item])

def show_input_all(dataloader):
    it = iter(dataloader)
    for i in range(len(dataloader)):
        X_batch, y_batch = next(it)
        num_elem_batch = 0
        for x_item, y_item in zip(X_batch, y_batch):
            show_input(x_item, title=class_names[y_item], num_batch=i+1, num_elem_batch=num_elem_batch+1)
            num_elem_batch += 1

# def show_input_all(dataloader):
#     i = 0
#     for X_batch, y_batch in dataloader:
#         num_elem_batch = 0
#         for x_item, y_item in zip(X_batch, y_batch):
#             show_input(x_item, title=class_names[y_item], num_batch=i+1, num_elem_batch=num_elem_batch+1)
#             num_elem_batch += 1
#         i += 1

#     i = 0
#     for X_batch, y_batch in dataloader:
#         num_elem_batch = 0
#         for x_item, y_item in zip(X_batch, y_batch):
#             show_input(x_item, title=class_names[y_item], num_batch=i+1, num_elem_batch=num_elem_batch+1)
#             num_elem_batch += 1
#         i += 1

In [ ]:
show_input_all(train_dataloader)

In [ ]:
show_input_all(val_dataloader)

In [26]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model

In [ ]:
model = models.resnet18(pretrained=True)

# Disable grad for all conv layers
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(model.fc.in_features, 2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(model)
print(device)
model = model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
train_model(model, loss, optimizer, scheduler, num_epochs=100)

In [29]:
test_dir = 'test'
# shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [30]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

test_dataset = ImageFolderWithPaths(test_dir, val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [31]:
test_dataset

Dataset ImageFolderWithPaths
    Number of datapoints: 744
    Root location: test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)

# test_predictions = np.concatenate(test_predictions)
# print(test_predictions)
# print(len(test_predictions))
# print(len(test_predictions[0]))

In [ ]:
# inputs, labels, paths = next(iter(test_dataloader))
# for img, pred in zip(inputs, test_predictions):
#     show_input(img, title=pred)

iter = iter(test_dataloader)
for i in range(len(test_dataloader)):
    inputs, labels, paths = next(iter)
    num_elem_batch = 0
    for img, pred in zip(inputs, test_predictions[i]):
        show_input(img, title=str(pred), num_batch=i+1, num_elem_batch=num_elem_batch+1)
        num_elem_batch += 1

test_predictions = np.concatenate(test_predictions)

In [35]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [ ]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace(test_dir + "\\unknown\\", "")
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

In [38]:
submission_df.to_csv('submission2.csv')

In [ ]:
# !rm -rf train val test